# Deep Learning for Depth Object Recognition
---
`Zhiang Chen, Wyatt Newman
Aug 2016`

https://www.youtube.com/watch?v=pJuiyjRpX_0

#### 1. import packages

In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import matplotlib.pyplot as plt
import random
import operator

#### 2. load data

In [5]:
file_name = 'depth_data'
with open(file_name, 'rb') as f:
    save = pickle.load(f)
    dataset = save['dataset']
    names = save['names']
    orientations = save['orientations']
    del save

#### 3. data preprocess

In [6]:
# generate labels
# for 10 objectives
image_size = 34
num_labels = 10
num_channels = 1

num_images = dataset.shape[0]
num_train = round(num_images*0.7)
num_valid = round(num_images*0.1)
num_test = round(num_images*0.2)

name2value = {'v8':0,'ducky':1,'stapler':2,'pball':3,'tball':4,'sponge':5,'bclip':6,'tape':7,'gstick':8,'cup':9}
value2name = dict((value,name) for name,value in name2value.items())    

labels = np.ndarray(num_images, dtype=np.int32)
index = 0
for name in names:
    labels[index] = name2value[name]
    index += 1
        
def randomize(dataset, labels):
    permutation = np.random.permutation(labels.shape[0])
    shuffled_dataset = dataset[permutation,:,:]
    shuffled_labels = labels[permutation]
    return shuffled_dataset, shuffled_labels

rdataset, rlabels = randomize(dataset, labels)
train_dataset = rdataset[0:num_train,:,:]
train_labels = rlabels[0:num_train]
valid_dataset = rdataset[num_train:(num_train+num_valid),:,:]
valid_labels = rlabels[num_train:(num_train+num_valid)]
test_dataset = rdataset[(num_train+num_valid):,:,:]
test_labels = rlabels[(num_train+num_valid):]
print('Training:', train_dataset.shape, train_labels.shape)
print('Validation:', valid_dataset.shape, valid_labels.shape)
print('Testing:', test_dataset.shape, test_labels.shape)

Training: (22680, 34, 34) (22680,)
Validation: (3240, 34, 34) (3240,)
Testing: (6480, 34, 34) (6480,)


display some images to verify that the data is still correct

In [7]:
indices = [random.randint(0,train_dataset.shape[0]) for x in range(3)]
for index in indices:
    image = train_dataset[index,:,:]
    print(value2name[train_labels[index]])
    plt.imshow(image,cmap='Greys_r')
    plt.show()

pball
ducky
tape


In [8]:
print('......Reformatting......')

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

......Reformatting......
Training set (22680, 34, 34, 1) (22680, 10)
Validation set (3240, 34, 34, 1) (3240, 10)
Test set (6480, 34, 34, 1) (6480, 10)


#### 4. Define Costumed Function

In [9]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

#### 5. Build NN

In [10]:
import time


batch_size = 16
patch_size = 5
kernel_size = 2
depth1 = 6 #the depth of 1st convnet
depth2 = 16 #the depth of 2nd convnet
C5_units = 120
F6_units = 84
F7_units = 10

graph = tf.Graph()

with graph.as_default():
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    # convolution's input is a tensor of shape [batch,in_height,in_width,in_channels]
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, 10))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables(weights and biases)
    C1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth1], stddev=0.1))
    # convolution's weights are called filter in tensorflow
    # it is a tensor of shape [kernel_hight,kernel_width,in_channels,out_channels]
    C1_biases = tf.Variable(tf.zeros([depth1]))
                            
    # S1_weights # Sub-sampling doesn't need weights and biases
    # S1_biases
    
    C3_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth1, depth2], stddev=0.1))
    C3_biases = tf.Variable(tf.constant(1.0, shape=[depth2]))
                            
    # S4_weights
    # S4_biases
     
    # C5 actually is a fully-connected layer                        
    C5_weights = tf.Variable(tf.truncated_normal([6 * 6 * depth2, C5_units], stddev=0.1))
    C5_biases = tf.Variable(tf.constant(1.0, shape=[C5_units]))
         
    F6_weights = tf.Variable(tf.truncated_normal([C5_units,F6_units], stddev=0.1))
    F6_biases = tf.Variable(tf.constant(1.0, shape=[F6_units]))
                                
    # FC and logistic regression replace RBF
    F7_weights = tf.Variable(tf.truncated_normal([F6_units,F7_units], stddev=0.1))
    F7_biases = tf.Variable(tf.constant(1.0, shape=[F7_units]))

    # Model
    def model(data):
        conv = tf.nn.conv2d(data, C1_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + C1_biases) # relu is better than tanh
        
        max_pool = tf.nn.max_pool(hidden,[1,kernel_size,kernel_size,1],[1,2,2,1],'VALID')
        hidden = tf.nn.relu(max_pool)
                                
        conv = tf.nn.conv2d(hidden, C3_weights, [1, 1, 1, 1], padding='VALID')
        hidden = tf.nn.relu(conv + C3_biases)

        max_pool = tf.nn.max_pool(hidden,[1,kernel_size,kernel_size,1],[1,2,2,1],'VALID')
        hidden = tf.nn.relu(max_pool)
                            
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, C5_weights) + C5_biases)
                            
        fc = tf.matmul(hidden,F6_weights)
        hidden = tf.nn.relu(fc + F6_biases)
        
        fc = tf.matmul(hidden,F7_weights)
        output = fc + F7_biases
    
        return output

    
    # Training computation.
    tf_train_dataset = tf.nn.dropout(tf_train_dataset,0.8) # input dropout
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.001).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

#### 6. Train NN

In [ ]:
## training
start_time = time.time()

num_steps = 15000
config = tf.ConfigProto()
config.log_device_placement = True
with tf.Session(graph=graph, config = config) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
      print('--------------------------------------')
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
  end_time = time.time()
  duration = (end_time - start_time)/60
  print("Excution time: %0.2fmin" % duration)
  i_test = 0
  while(i_test!=''):
        i_test = input("Input an index of test image (or Enter to quit): ")
        label = test_labels[int(i_test),:].tolist()
        #print("Correct label: "+value2name[label.index(1)])
        image = test_dataset[int(i_test),:,:,:].reshape((-1,image_size,image_size,num_channels)).astype(np.float32)
        prediction = tf.nn.softmax(model(image))
        pre_dict = dict(zip(list(range(9)),prediction.eval()[0]))
        sorted_pre_dict = sorted(pre_dict.items(), key=operator.itemgetter(1))
        name1 = value2name[sorted_pre_dict[-1][0]]
        value1 = str(sorted_pre_dict[-1][1])
        name2 = value2name[sorted_pre_dict[-2][0]]
        value2 = str(sorted_pre_dict[-2][1])
        tile = name1+': '+value1+'\n'+name2+': '+value2
        image = image.reshape((image_size,image_size)).astype(np.float32)
        plt.imshow(image,cmap='Greys_r')
        plt.suptitle(tile, fontsize=12)
        plt.xlabel(value2name[label.index(1)], fontsize=12)
        plt.show()
        

Initialized
Minibatch loss at step 0: 2.895051
Minibatch accuracy: 12.5%
Validation accuracy: 13.8%
--------------------------------------
Minibatch loss at step 1000: 2.194652
Minibatch accuracy: 18.8%
Validation accuracy: 29.6%
--------------------------------------
Minibatch loss at step 2000: 1.423794
Minibatch accuracy: 50.0%
Validation accuracy: 61.5%
--------------------------------------
Minibatch loss at step 3000: 0.707042
Minibatch accuracy: 81.2%
Validation accuracy: 64.2%
--------------------------------------
Minibatch loss at step 4000: 0.555287
Minibatch accuracy: 93.8%
Validation accuracy: 80.7%
--------------------------------------
Minibatch loss at step 5000: 0.404038
Minibatch accuracy: 100.0%
Validation accuracy: 85.8%
--------------------------------------
Minibatch loss at step 6000: 0.250627
Minibatch accuracy: 87.5%
Validation accuracy: 90.0%
--------------------------------------
Minibatch loss at step 7000: 0.338935
Minibatch accuracy: 81.2%
Validation accur